In [18]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd
from faker import Faker
fake = Faker()

In [ ]:
engine = create_engine(connstr)
PK_FORM = '2021-Q4-BOD2201'
Acronym='GSA'
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [25]:
query = f""" \
    SELECT TOP 5000 * FROM vwQuestions WHERE PK_FORM LIKE '%{PK_FORM}%'
"""  
df_vwQuestions = get_df(query)

query = f""" \
    SELECT TOP 5000 * FROM  vwOrgSubToComponent WHERE PK_FORM LIKE '%{PK_FORM}%' AND Acronym = '{Acronym}'
    """  
df_vwOrgSubToComponent = get_df(query)

query = f""" \
    SELECT TOP 5000 * FROM  vwMetricsAnswers WHERE PK_FORM LIKE '%{PK_FORM}%' AND Acronym = '{Acronym}'
    """  
df_vwMetricsAnswers = get_df(query) 

In [ ]:
for i, r in df_vwQuestions.iterrows():
    s = ''
    s = s + f" -- {str(r['PK'])}  {str(r['QGROUP'])}"
    print( s ) 
 

In [ ]:
###################################################################################

In [ ]:
conn = engine.connect()
query = """ \
SELECT TOP 5000 
* 
FROM CVEUnremediated
WHERE PK_OrgSubmission = 29150 
"""  
df = pd.read_sql(query,con=conn) 
conn.close()
df_CVEUN=df

In [17]:
cols = ', '.join(df_CVEUN.columns[1:])
cols

'PK_OrgSubmission, PK_CISA_CVE, UnremediatedCount, Constraints, UserId, DateUploaded, DateModified, isActive, CveCountJustification'

In [ ]:
cols = 'PK_OrgSubmission, PK_CISA_CVE, UnremediatedCount, Constraints, UserId, DateUploaded, DateModified, isActive, CveCountJustification'
cols

In [16]:
vals = ''
h = 'INSERT INTO CVEUnremediated (PK_OrgSubmission, PK_CISA_CVE, UnremediatedCount, Constraints, UserId, DateUploaded, DateModified, isActive, CveCountJustification)'
h = h + '\nVALUES\n'
for i, r in df_CVEUN.iterrows(): 
    s = ''
    for c in df_CVEUN.columns[1:]: 
        s = s + f"{  r[c]  }," 
    vals = vals + f'({ s[:-1] }),\n'     
print( h ) 

vals = vals[:-1]
vals = vals.replace("None", "NULL") 
vals = re.sub(r"(CVE\-\d+\-\d+)", "'\\1'", vals)

print( vals ) 

INSERT INTO CVEUnremediated (PK_OrgSubmission, PK_CISA_CVE, UnremediatedCount, Constraints, UserId, DateUploaded, DateModified, isActive, CveCountJustification)
VALUES

(29150,'CVE-2002-0367',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2004-0210',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2005-2773',2,,1169,NULL,NULL,True,NULL),
(29150,'CVE-2006-1547',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2008-2992',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2008-3431',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2009-0927',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2009-1123',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2009-1151',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2009-2055',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2009-3129',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2009-3960',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2010-0188',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2010-0232',0,NULL,1169,NULL,NULL,True,NULL),
(29150,'CVE-2010-1871',0,NULL,1169,NU